In [ ]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# GRU
class GRURNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size, output_size)
        self.fc2 = nn.Linear(self.hidden_size, output_size)
        self.fc3 = nn.Linear(self.hidden_size, output_size)
        self.fc4 = nn.Linear(self.hidden_size, output_size)
        self.fc5 = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, input_seq):
        batch_size = input_seq.shape[0]
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        output, _ = self.gru(input_seq,h_0)
        pred1 = self.fc1(output)
        pred2 = self.fc2(output)
        pred3 = self.fc3(output)
        pred4 = self.fc4(output)
        pred5 = self.fc5(output)
        pred1, pred2, pred3, pred4, pred5 = pred1[:, -1, :], pred2[:, -1, :], pred3[:, -1, :], pred4[:, -1, :], pred5[:, -1, :]
        pred1, pred2, pred3, pred4, pred5 = self.softmax(pred1), self.softmax(pred2), self.softmax(pred3), self.softmax(pred4), self.softmax(pred5)
        pred = torch.stack([pred1, pred2, pred3, pred4, pred5], dim=1)
        return pred

In [ ]:
class Config():
    data_path = './matchstat.csv' # TODO: change data_path
    timestep = 10
    batch_size = 32
    feature_size = 13 # TODO: change the feature size accordingly
    hidden_size = 256
    output_size = 4
    num_layers = 5 # TODO: can change
    epochs = 10 # TODO: can change
    best_loss = 0
    learning_rate = 0.0003 # TODO: can change
    model_name = 'GRU_for_tennis_momentum_swing'
    save_path = './{}.pth'.format(model_name)

config = Config()

In [ ]:
df = pd.read_csv(config.data_path)
data = np.array(df)

def split_data(data, timestep, feature_size, output_size):
    dataX = []
    dataY = []
    for i in range(len(data) - timestep):
        dataX.append(data[i: i + timestep][:, :feature_size])


    train_size = np.round(0.8 * dataX.shape[0])
    trainX = dataX[:train_size, :].reshape(-1, timestep, feature_size)
    testX = dataX[train_size:, :].reshape(-1, timestep, feature_size)
    trainY = dataY[:train_size, :].reshape(-1, output_size)
    testY = dataY[train_size:, :].reshape(-1, output_size)
    return trainX, trainY, testX, testY

trainX, trainY, testX, testY = split_data(data, config.timestep, config.feature_size, config.output_size)
x_train_tensor = torch.from_numpy(trainX).to(torch.float32)
y_train_tensor = torch.from_numpy(trainY).to(torch.float32)
x_test_tensor = torch.from_numpy(testX).to(torch.float32)
y_test_tensor = torch.from_numpy(testY).to(torch.float32)

train_data = TensorDataset(x_train_tensor, y_train_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, config.batch_size, False)
test_loader = DataLoader(test_data, config.batch_size, False)


In [ ]:
model = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [ ]:
for epoch in range(config.epochs):
    model.train()
    running_loss = 0
    train_bar = tqdm(train_loader)
    for data in train_bar:
        x_train, y_train = data
        optimizer.zero_grad()
        y_train_pred = model(x_train)
        loss = loss_function(y_train_pred, y_train.reshape(-1, config.output_size))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.desc = 'train epoch[{}/{}] loss: {:.3f}'.format(epoch + 1, config.epochs, loss)


In [2]:
import torch
import torch.nn as nn
import math

entroy=nn.CrossEntropyLoss()
input=torch.Tensor([[-0.7715, -0.6205,-0.2562], [-0.7715, -0.6205,-0.2562]])
target = torch.tensor([0, 1])
output = entroy(input, target)
print(output)

tensor(1.2692)
